# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
geoapify_key 

'72a8ab71452543b28621184a5b9be380'

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,taroa,8.7058,171.2275,26.17,91,100,6.34,MH,1699577473
1,1,hirara,24.8000,125.2833,27.15,83,40,3.60,JP,1699577473
2,2,port alfred,-33.5906,26.8910,16.35,85,0,2.52,ZA,1699577473
3,3,papatowai,-46.5619,169.4708,12.28,48,5,2.94,NZ,1699577473
4,4,wailua homesteads,22.0669,-159.3780,26.44,80,20,3.58,US,1699577474


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points('Lng', 'Lat', geo = True, tiles = 'OSM', s = 'Humidity', color='City', frame_height=500, frame_width=800)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal = city_data_df.loc[(city_data_df['Max Temp']<=30) &
                             (city_data_df['Max Temp']>=5) &
                             (city_data_df['Cloudiness']<=50) &
                             (city_data_df['Humidity']<=50) &
                             (city_data_df['Wind Speed'] <=5)]

# Drop any rows with null values
my_ideal.dropna()

# Display sample data
my_ideal.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,papatowai,-46.5619,169.4708,12.28,48,5,2.94,NZ,1699577473
28,28,hotan,37.0997,79.9269,5.09,26,0,2.48,CN,1699577482
41,41,port lincoln,-34.7333,135.8667,26.90,47,10,2.37,AU,1699577485
99,99,bom despacho,-19.7364,-45.2522,21.92,35,0,2.26,BR,1699577500
119,119,hamilton,39.1834,-84.5333,13.74,44,0,4.12,US,1699577162


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,papatowai,NZ,-46.5619,169.4708,48,
28,hotan,CN,37.0997,79.9269,26,
41,port lincoln,AU,-34.7333,135.8667,47,
99,bom despacho,BR,-19.7364,-45.2522,35,
119,hamilton,US,39.1834,-84.5333,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting Hotel Search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address_get = requests.get(base_url, params=params)
    print(name_address_get.text)
    # Convert the API response to JSON format
    name_address = name_address_get.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting Hotel Search
{"type":"FeatureCollection","features":[]}
papatowai - nearest hotel: No hotel found
{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Hua Yi International Hotel (accepts foreigners)","country":"China","country_code":"cn","region":"Hotan Prefecture","state":"Xinjiang","city":"Hotan City","town":"Gujanbagh","suburb":"Gulebage Subdistrict","street":"南环路","lon":79.9250378,"lat":37.0995874,"formatted":"Hua Yi International Hotel (accepts foreigners), 南环路, Gulebage Subdistrict, Xinjiang, China","address_line1":"Hua Yi International Hotel (accepts foreigners)","address_line2":"南环路, Gulebage Subdistrict, Xinjiang, China","categories":["accommodation","accommodation.hotel"],"details":["details"],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database Licence","url":"https://www.openstreetmap.org/copyright","raw":{"osm_id":6567694985,"name:en":"Hua Yi International Hotel (accepts foreign

,City,Country,Lat,Lng,Humidity,Hotel Name
3,papatowai,NZ,-46.5619,169.4708,48,No hotel found
28,hotan,CN,37.0997,79.9269,26,Hua Yi International Hotel (accepts foreigners)
41,port lincoln,AU,-34.7333,135.8667,47,Boston Hotel
99,bom despacho,BR,-19.7364,-45.2522,35,Letícia Hotel
119,hamilton,US,39.1834,-84.5333,44,North Vista Manor


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', s='Humidity', frame_height=500, frame_width=800, color='City')

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)